# Feature Extraction and Processing for GoodReads Dataset
## Pre-processing input data, creating Embeddings, etc.

### Setup

In [2]:
import json
import os
import numpy as np
import pandas as pd
from IPython.display import display
import pprint
import tensorflow as tf

Dataset Directory

In [3]:
DIR = './data/cleaned/'
fn_books = 'goodreads_books_mystery_thriller_crime.pkl'
fn_reviews = 'goodreads_reviews_mystery_thriller_crime.pkl'

In [4]:
df_reviews = pd.read_pickle(os.path.join(DIR, fn_reviews))
df_books = pd.read_pickle(os.path.join(DIR, fn_books))

In [5]:
df_reviews.head()

,user_id,book_id,rating,review_text
review_id,,,,
5e212a62bced17b4dbe41150e5bb9037,8842281e1d1347389f2ab93d60773d4d,6392944,3,I haven't read a fun mystery book in a while a...
2ede853b14dc4583f96cf5d120af636f,8842281e1d1347389f2ab93d60773d4d,28684704,3,"A fun, fast paced science fiction thriller. I ..."
8e4d61801907e591018bdc3442a9cf2b,8842281e1d1347389f2ab93d60773d4d,32283133,0,http://www.telegraph.co.uk/culture/10...
022bb6daffa49adc27f6b20b6ebeb37d,8842281e1d1347389f2ab93d60773d4d,17860739,4,An amazing and unique creation: JJ Abrams and ...
0e317947e1fd341f573192111bb2921d,8842281e1d1347389f2ab93d60773d4d,8694005,3,The Name of the Rose is a thrilling Dan Brown-...


In [19]:
df_books.rename(columns={"name": "author_name"}, inplace=True)
df_books.head()

,title,text_reviews_count,popular_shelves,average_rating,description,author_id,author_name
book_id,,,,,,,
6066814,"Crowner Royal (Crowner John Mystery, #13)",15,"[{'count': '159', 'name': 'to-read'}, {'count'...",3.93,"London, 1196. At the command of Richard the Li...",37778,Bernard Knight
33394837,The House of Memory (Pluto's Snitch #2),60,"[{'count': '54', 'name': 'currently-reading'},...",4.33,,242185,Carolyn Haines
29074697,The Slaughtered Virgin of Zenopolis (Inspector...,23,"[{'count': '90', 'name': 'to-read'}, {'count':...",3.49,"BATHS, BANKS AND ROMAN INSURRECTION\nDetective...",15104629,David Blake
1902202,"Dead in the Morning (Patrick Grant, #1)",8,"[{'count': '51', 'name': 'to-read'}, {'count':...",3.30,"Gerald breezily introduced his wife, Helen, to...",190988,Margaret Yorke
9671977,Aristotele e i misteri di Eleusi,3,"[{'count': '48', 'name': 'to-read'}, {'count':...",3.54,"""I misteri di Eleusi"" e il quinto romanzo di A...",337108,Margaret Doody


In [20]:
df_join = pd.merge(df_reviews, df_books, left_on="book_id", right_index=True)
df_join.head()

,user_id,book_id,rating,review_text,title,text_reviews_count,popular_shelves,average_rating,description,author_id,author_name
review_id,,,,,,,,,,,
d44b6d28f456dbcc5b1537b4c5572400,8842281e1d1347389f2ab93d60773d4d,6411961,3,"** spoiler alert ** \n Critics aside, Dan Brow...","The Lost Symbol (Robert Langdon, #3)",21569,"[{'count': '9279', 'name': 'currently-reading'...",3.66,WHAT IS LOST...\nWILL BE FOUND\nIn this stunni...,630,Dan Brown
41e98a56ec0abd77d9191b4f1e2cd806,40b795551cd5b1dae85ff2f7b8ddee27,6411961,2,"While the book is a page-turner, I found the p...","The Lost Symbol (Robert Langdon, #3)",21569,"[{'count': '9279', 'name': 'currently-reading'...",3.66,WHAT IS LOST...\nWILL BE FOUND\nIn this stunni...,630,Dan Brown
9ccb9f3b2719283bd8a6d24c297cd738,2193b39163317f53641d1f8539a4586c,6411961,3,As always well written and you have to keep re...,"The Lost Symbol (Robert Langdon, #3)",21569,"[{'count': '9279', 'name': 'currently-reading'...",3.66,WHAT IS LOST...\nWILL BE FOUND\nIn this stunni...,630,Dan Brown
a4f43a0a2703c9c46a18ba1b8a7a2b90,21e67f8f9a1679070e93ff8a152efde6,6411961,3,"I am sorry to say this, but i found the plot o...","The Lost Symbol (Robert Langdon, #3)",21569,"[{'count': '9279', 'name': 'currently-reading'...",3.66,WHAT IS LOST...\nWILL BE FOUND\nIn this stunni...,630,Dan Brown
0b8368d5286c6ebf3a22a7ed3572b513,1939a0e8cacc2101a1f6d5446d91de78,6411961,4,"Book two - loved it, couldn't put down until I...","The Lost Symbol (Robert Langdon, #3)",21569,"[{'count': '9279', 'name': 'currently-reading'...",3.66,WHAT IS LOST...\nWILL BE FOUND\nIn this stunni...,630,Dan Brown


In [34]:
df_join.describe()

,book_id,rating,text_reviews_count,average_rating,author_id
count,4.272000e+03,4272.000000,4272.000000,4272.000000,4.272000e+03
mean,1.275792e+07,3.711376,1806.218633,3.861505,2.245762e+06
std,1.068043e+07,1.168053,4594.836138,0.280653,3.726043e+06
min,2.300000e+02,0.000000,1.000000,2.530000,1.310000e+02
25%,1.038334e+06,3.000000,33.000000,3.690000,1.375700e+04
50%,1.271421e+07,4.000000,193.000000,3.890000,1.448005e+05
75%,2.146169e+07,5.000000,1001.750000,4.050000,3.418028e+06
max,3.620798e+07,5.000000,24868.000000,4.750000,1.703360e+07


In [10]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 6066814 to 23960943
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               30000 non-null  object 
 1   text_reviews_count  30000 non-null  uint32 
 2   popular_shelves     30000 non-null  object 
 3   average_rating      30000 non-null  float64
 4   description         30000 non-null  object 
 5   author_id           30000 non-null  int64  
 6   name                16228 non-null  object 
dtypes: float64(1), int64(1), object(4), uint32(1)
memory usage: 1.7+ MB


### Join Reviews and Book Infos
#### only 4272 book-review pairs are overlapping in total

In [11]:
df_join[100:105]

,user_id,book_id,rating,review_text,title,text_reviews_count,popular_shelves,average_rating,description,author_id,name
review_id,,,,,,,,,,,
fe2b43f4dcacee1bd5e14433d19139f1,70eb5a06d5aa23ecc694850b980309a9,5060378,4,"Really liked this book, though I can never fig...","The Girl Who Played with Fire (Millennium, #2)",24868,"[{'count': '6613', 'name': 'fiction'}, {'count...",4.22,The Expose\nMillenniumpublisher Mikael Blomkvi...,706255,Stieg Larsson
87f62eb9a9d4d41f57125463b62066e2,4067b56ccfa82b34e0ea0bfa2cc99cf1,5060378,5,I loved The Girl With The Dragon Tattoo and I ...,"The Girl Who Played with Fire (Millennium, #2)",24868,"[{'count': '6613', 'name': 'fiction'}, {'count...",4.22,The Expose\nMillenniumpublisher Mikael Blomkvi...,706255,Stieg Larsson
7dc1dfe51c0e26d24d3d16fbe7c06c6d,d8c86e725d683042e9eec675f294d7d4,5060378,5,One of my favorite books. Ever. \n This book i...,"The Girl Who Played with Fire (Millennium, #2)",24868,"[{'count': '6613', 'name': 'fiction'}, {'count...",4.22,The Expose\nMillenniumpublisher Mikael Blomkvi...,706255,Stieg Larsson
4f8957565f264bbea508417dfeb0f793,ab2fadb5c7bbe55c80406d2b3692e969,5060378,3,Argh! This book kind of killed me. For a good ...,"The Girl Who Played with Fire (Millennium, #2)",24868,"[{'count': '6613', 'name': 'fiction'}, {'count...",4.22,The Expose\nMillenniumpublisher Mikael Blomkvi...,706255,Stieg Larsson
8180c6416aa339537abbe5049c330128,246eac0ca4d02e73d8768a5acb9b89d7,5060378,3,"This series remains a guilty pleasure, though ...","The Girl Who Played with Fire (Millennium, #2)",24868,"[{'count': '6613', 'name': 'fiction'}, {'count...",4.22,The Expose\nMillenniumpublisher Mikael Blomkvi...,706255,Stieg Larsson


In [12]:
df_join.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4272 entries, d44b6d28f456dbcc5b1537b4c5572400 to 041fa5ebdb3e1c8f70ff67b1ea689031
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             4272 non-null   object 
 1   book_id             4272 non-null   uint32 
 2   rating              4272 non-null   uint8  
 3   review_text         4272 non-null   object 
 4   title               4272 non-null   object 
 5   text_reviews_count  4272 non-null   uint32 
 6   popular_shelves     4272 non-null   object 
 7   average_rating      4272 non-null   float64
 8   description         4272 non-null   object 
 9   author_id           4272 non-null   int64  
 10  name                2886 non-null   object 
dtypes: float64(1), int64(1), object(6), uint32(2), uint8(1)
memory usage: 337.9+ KB


Reviews + Books joinen ? für Titel usw. ?

In [13]:
book_title_lookup = tf.keras.layers.StringLookup()

In [14]:
book_title_lookup.adapt(df_books['title'])
print(f"Vocabulary: {book_title_lookup.get_vocabulary()[:5]}")

Vocabulary: ['[UNK]', 'The Woman in White', 'The Adventures of Sherlock Holmes', 'Rebecca', 'A Study in Scarlet']


now we can translate raw tokens (title) into embedding ids (here -> 1)

In [15]:
book_title_lookup("The Woman in White")


<tf.Tensor: shape=(), dtype=int64, numpy=1>

you have OOV (out of vocabulary) for unknown tokens. StringLookup can use multiple OOV indices. the more indices, the less likely two different feature values will hash to the same OOV index. To take it to the next level, let's _just_ use hashing

In [16]:
# We set up a large number of bins to reduce the chance of hash collisions.
num_hashing_bins = 200_000

book_title_hashing = tf.keras.layers.Hashing(
    num_bins=num_hashing_bins
)


In [17]:
book_title_hashing(['The Woman in White'])

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([145026], dtype=int64)>

Now let's turn those integers into **Embeddings**
> An embedding layer has two dimensions: the first dimension tells us how many distinct categories we can embed; the second tells us how large the vector representing each of them can be. When creating the embedding layer for movie titles, we are going to set the first value to the size of our title vocabulary (or the number of hashing bins). The second is up to us: the larger it is, the higher the capacity of the model, but the slower it is to fit and serve.


In [18]:
book_title_embedding = tf.keras.layers.Embedding(
    # Let's use the explicit vocabulary lookup. (for whatever reason..)
    input_dim=book_title_lookup.vocabulary_size(),
    output_dim=32
)

String -> Integer -> Embedding

In [40]:
book_title_model = tf.keras.Sequential([tf.keras.Input(shape=(1,), dtype=tf.string), book_title_lookup, book_title_embedding])

In [43]:
book_title_model.predict(['The Woman in White'])

array([[[-0.04795183,  0.04396185,  0.04338274,  0.01306342,
         -0.0034074 , -0.02287626,  0.00849246,  0.0158684 ,
          0.01486142,  0.01317045,  0.02869229,  0.03591696,
         -0.01345669,  0.00844286,  0.0126904 , -0.04739565,
          0.02517379,  0.01828475,  0.02442754,  0.02419094,
         -0.03021648,  0.03639397,  0.03049425,  0.01627653,
         -0.00817373, -0.01427885,  0.04532106, -0.03992269,
         -0.0232616 ,  0.04623466,  0.00218357,  0.00812031]]],
      dtype=float32)

Jetzt das gleiche mit User IDs 
--> nicht nötig, da sie schon pure ints sind.

### Normalize Ratings
not sure if necessary, but hey, doesn't hurt.
not sure if min/max rescaling (0 - 1) or normalization is better...

In [72]:
rating_scaling = tf.keras.layers.Rescaling(scale=1/5.)

In [74]:
rating_normalization = tf.keras.layers.Normalization()
rating_normalization.adapt(df_reviews['rating'])
for x in df_reviews['rating'].sample(5):
    print(f"normalized rating: {rating_normalization(x)}, was {x}")

normalized rating: [[0.26640868]], was 4
normalized rating: [[1.1069033]], was 5
normalized rating: [[-0.57408595]], was 3
normalized rating: [[-0.57408595]], was 3
normalized rating: [[-0.57408595]], was 3


jetzt hab ich halt Kontinuerliche Werte aus Diskreten gemacht. das ist eigentlich dumm.

In [75]:
rating_scaling(df_reviews['rating'].sample())

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.8], dtype=float32)>

Ich bleib bei den rescaled Ratings. ist dann schön 0 - 1 aber auch nicht kategorisch, weil ja durchaus ein linearer Zusammenhang besteht.

### Processing text features

## TODO: nur englische Bücher verwenden!

In [46]:
description_text = tf.keras.layers.TextVectorization()
description_text.adapt(df_books['description'])


In [54]:
sample = df_books['description'].sample()
sample

book_id
18070389    When the trail of America's most wanted terror...
Name: description, dtype: object

In [55]:
description_text(sample)

<tf.Tensor: shape=(1, 71), dtype=int64, numpy=
array([[   22,     2,   391,     5,  1260,    64,   781,  1079,   318,
          424,     7, 34950,     4, 35886,    21,     3,  2108,  7365,
            8, 38497,     7,   702,  5642,    27,    55,    20,     3,
         9129,     2,  2076,  1079,     4,     2, 42729,    29,  1333,
            4,    36,  1532,  2675,    46,     7,     2,   284,   236,
         1291,  4454,   672,    14,   141,    98,    11,   236,     8,
            2,    52,   546,     8,   903,    14,    54,    62,    70,
          185,     7,    68,     6,  1510,    19,  2966,  4091]],
      dtype=int64)>

In [57]:
description_text.get_vocabulary()[35:45]


['into', 'will', 'they', 'him', 'have', 'life', 'e', 'all', 'out', 'its']

> To finish the processing, we now need to embed the text. Because each title contains multiple words, we will get multiple embeddings for each title. For use in a donwstream model these are usually compressed into a single embedding. Models like RNNs or Transformers are useful here, but averaging all the words' embeddings together is a good starting point.

## User Model

In [80]:
user_id_lookup = tf.keras.layers.StringLookup()
user_id_lookup.adapt(df_reviews["user_id"])

user_id_embedding = tf.keras.layers.Embedding(user_id_lookup.vocabulary_size(), 32)

user_id_model = tf.keras.Sequential([tf.keras.Input(shape=(1,), dtype=tf.string), user_id_lookup, user_id_embedding])


außer ID hab ich keine Infos zum User. könnte noch die Timestamps hinzufügen, ja. will ich aber erstmal nicht.

In [84]:
for row in df_reviews['user_id'].sample(2):
  print(f"Computed representations: {user_id_model.predict([row])}")


Computed representations: [[[ 0.03838258  0.0334564   0.03454919  0.00372748  0.03165824
    0.00075807  0.04269307 -0.04693964  0.01979567 -0.00531821
   -0.02700326  0.02265919  0.01576631  0.02158827  0.02764842
   -0.01916559 -0.03063707  0.02025689 -0.01541857  0.02078635
    0.02063136 -0.02356881  0.02482804  0.01814339 -0.00089605
   -0.0103486   0.00904109 -0.02392753  0.04720285  0.01891253
    0.02500046 -0.04277949]]]
Computed representations: [[[ 0.0410802   0.03618349 -0.00764035 -0.03487115  0.00224654
    0.03541446  0.03207782  0.02575357  0.00630678 -0.04194795
    0.04436341 -0.03732288  0.0157108  -0.01017121  0.03041244
   -0.00320935  0.03131555 -0.00314027  0.03207007  0.03056103
    0.02446505 -0.02898942  0.02729616 -0.03279417 -0.03488356
   -0.0249534  -0.00356615  0.00310653  0.01982636  0.00428434
   -0.01835685 -0.00186362]]]


## Book Model

In [50]:
class BookModel(tf.keras.Model):

  def __init__(self, title_vocabulary, description_vocabulary, author_vocabulary):
    super().__init__()

    max_tokens = 10_000
    embedding_dim = 32

    title_vectorization_layer = tf.keras.layers.TextVectorization(max_tokens=max_tokens)
    title_vectorization_layer.adapt(title_vocabulary)
    
    self.title_embedding = tf.keras.Sequential([
      title_vectorization_layer,
      tf.keras.layers.Embedding(max_tokens, embedding_dim, mask_zero=True),
      # We average the embedding of individual words to get one embedding vector
      # per description.
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    description_text_vectorization_layer = tf.keras.layers.TextVectorization(
      max_tokens=max_tokens)
    description_text_vectorization_layer.adapt(description_vocabulary)
    
    self.description_text_embedding = tf.keras.Sequential([
      description_text_vectorization_layer,
      tf.keras.layers.Embedding(max_tokens, embedding_dim, mask_zero=True),
      # We average the embedding of individual words to get one embedding vector
      # per description.
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    # author IDs are numerical
    #from sample list of author ids create a continous range
    # train an embedding representation of this range
    author_id_lookup = tf.keras.layers.IntegerLookup()
    author_id_lookup.adapt(author_vocabulary)

    self.author_id_embedding = tf.keras.Sequential([
      tf.keras.Input(shape=(1,), dtype=tf.string), 
      author_id_lookup,
      tf.keras.layers.Embedding(author_id_lookup.vocabulary_size(), embedding_dim)
      ])


  def call(self, inputs):
    #TODO
    return tf.concat([
        self.title_embedding(inputs["title"]),
        self.description_text_embedding(inputs["description"]),
        self.author_id_embedding(inputs["author_id"]),
    ], axis=1)


In [51]:
book_model = BookModel(
    title_vocabulary = df_books['title'],
    description_vocabulary = df_books['description'],
    author_vocabulary=df_books['author_id'])

In [67]:
sample = df_books[['title', 'description', 'author_id']].sample(1)
sample

,title,description,author_id
book_id,,,
23472496,"Mansion Mayhem (Shannon Porter Mystery, #2)",Shannon Porter has always loved a good ghost s...,9811235


In [68]:
book_model(sample)

<tf.Tensor: shape=(1, 96), dtype=float32, numpy=
array([[ 0.01112771, -0.02709219, -0.01553751, -0.0068041 , -0.01099528,
         0.02012353, -0.01005515, -0.00724117, -0.01050504,  0.00671815,
         0.00914775, -0.00264367, -0.00535824, -0.00324484,  0.00817143,
        -0.0174779 ,  0.01963048,  0.00199371, -0.01798421,  0.00387764,
         0.00444162, -0.02078277,  0.01504769,  0.00630839,  0.01237988,
        -0.00411349,  0.0026515 ,  0.01585814,  0.00269544, -0.01237703,
         0.00661305,  0.00057801, -0.00974381,  0.00555293,  0.00236249,
         0.00274014,  0.00246889, -0.00261118, -0.00493944,  0.00274934,
        -0.00255168, -0.00225754,  0.0011858 , -0.00278818, -0.00554439,
         0.00172749,  0.0024313 , -0.0013643 , -0.00522135,  0.00293299,
        -0.00129832,  0.00034169, -0.0032865 ,  0.00673993,  0.00232636,
         0.00624023, -0.00477741, -0.00309413, -0.00058002,  0.00196072,
         0.00385022,  0.00727834, -0.00454859,  0.00194733,  0.00169499,
  

## Review Model

In [74]:
class ReviewModel(tf.keras.Model):

  def __init__(self, user_ids, review_ids, book_ids):
    super().__init__()

    max_tokens = 10_000
    embedding_dim = 32

    # Book IDs are numerical
    #from sample list of book ids create a continous range
    # train an embedding representation of this range
    book_id_lookup = tf.keras.layers.IntegerLookup()
    book_id_lookup.adapt(book_ids)

    self.book_id_embedding = tf.keras.Sequential([
      tf.keras.Input(shape=(1,), dtype=tf.string), 
      book_id_lookup,
      tf.keras.layers.Embedding(book_id_lookup.vocabulary_size(), embedding_dim)
      ])

    # Review and User IDs are alphanumerical
    
    review_id_vectorization_layer = tf.keras.layers.TextVectorization(
      max_tokens=max_tokens)
    review_id_vectorization_layer.adapt(review_ids)

    self.review_id_embedding = tf.keras.Sequential([
      review_id_vectorization_layer,
      tf.keras.layers.Embedding(review_id_vectorization_layer.vocabulary_size(), embedding_dim, mask_zero=True),
      # We average the embedding of individual words to get one embedding vector
      # per description.
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    user_id_vectorization_layer = tf.keras.layers.TextVectorization(
      max_tokens=max_tokens)
    user_id_vectorization_layer.adapt(user_ids)

    self.user_id_embedding = tf.keras.Sequential([
      user_id_vectorization_layer,
      tf.keras.layers.Embedding(user_id_vectorization_layer.vocabulary_size(), embedding_dim, mask_zero=True),
      # We average the embedding of individual words to get one embedding vector
      # per description.
      tf.keras.layers.GlobalAveragePooling1D(),
    ])


  def call(self, inputs: pd.DataFrame):
    #TODO
    if 'review_id' in inputs.keys():
      review_input = inputs['review_id']
    elif inputs.index.name == 'review_id':
      review_input = inputs.index
    else:
      raise ValueError("'review_id' neither index nor column")
    return tf.concat([
        self.book_id_embedding(inputs["book_id"]),
        self.review_id_embedding(review_input),
        self.user_id_embedding(inputs["user_id"]),
    ], axis=1)


In [75]:
review_model = ReviewModel(
    user_ids = df_reviews['user_id'],
    review_ids = df_reviews.index,
    book_ids=df_reviews['book_id'])

In [76]:
sample = df_reviews.sample(1)
sample

,user_id,book_id,rating,review_text
review_id,,,,
70da2109e9ba3003890d695be97ac100,30856098a61476bee91b3a8c41c49be4,11351241,4,Dead of Winter: A Kate Jones Thriller - A Book...


In [77]:
review_model(sample)

<tf.Tensor: shape=(1, 96), dtype=float32, numpy=
array([[ 0.00607531,  0.03227402, -0.0362423 ,  0.04253658, -0.04894707,
         0.00664612,  0.04461039, -0.03653277,  0.03647492,  0.01883667,
         0.00467142,  0.03029758,  0.03740216, -0.00269256, -0.0109259 ,
        -0.03355049, -0.04445238, -0.01009589, -0.03650501,  0.01309821,
         0.02085717, -0.0121031 , -0.00135223,  0.02748163, -0.00403132,
        -0.01134054,  0.02568194, -0.00377088,  0.01700332, -0.02662518,
        -0.04688637,  0.01247954,  0.03484925,  0.01412565, -0.04374253,
        -0.0138613 , -0.0367738 , -0.00228997, -0.04909359, -0.02015574,
        -0.011106  ,  0.03455187, -0.03977926, -0.0223038 , -0.01922525,
        -0.01144892,  0.04210733,  0.02345494, -0.00821276,  0.045091  ,
        -0.03539561, -0.00963514,  0.00915825,  0.00831991,  0.03260373,
         0.04554411,  0.02585772, -0.03348044,  0.00241506,  0.0387742 ,
        -0.0401826 , -0.01237367, -0.03643153, -0.04797146,  0.01347378,
  

## TODO: Review ID kann vermutlich weg?
### bzw. brauch ich das ReviewModel *überhaupt*?